In [40]:
import scipy.stats as stats

In [10]:
# generate match SNP sets
os.chdir("/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/sig/bed/SNPsnap/")

In [11]:
file_list = glob.glob("SNPsnap_*/matched_snps.txt")
for x in file_list:
    result_file = x.replace("txt",'bed')
    with open(result_file, 'w') as fw:
        with open(x, 'r') as f:
            f.readline()
            content = f.readlines()
            for line in content:
                snp_list = line.strip().split("\t")
                query_snp, remain_snp = snp_list[0], snp_list[1:]
                a = query_snp.split(":")[0] + ":"
                condition = 0
                for y in remain_snp:
                    if y.startswith(a):
                        chr_y,pos_y = "chr" + y.split(":")[0], y.split(":")[1].strip()
                        fw.write("%s\t%s\t%s\n" % (chr_y, pos_y, pos_y))
                        condition = 1
                        break
                if condition == 0:
                    y=remain_snp[0]
                    chr_y,pos_y = "chr" + y.split(":")[0], y.split(":")[1].strip()
                    fw.write("%s\t%s\t%s\n" % (chr_y, pos_y, pos_y))

In [37]:
# process cluster
efo = "/home/database/GWAS/gwas_catalog_trait-mappings_r2020-05-03.tsv"
df = pd.read_table(efo)
print(df.head())
print(df.columns)

                                       Disease trait                  ...                                             Parent URI
0                                           Clubfoot                  ...                   http://www.ebi.ac.uk/efo/EFO_0000408
1  Alcohol dependence or heroin dependence or met...                  ...                   http://www.ebi.ac.uk/efo/EFO_0000618
2                         Methamphetamine dependence                  ...                   http://www.ebi.ac.uk/efo/EFO_0000618
3                                     Hypothyroidism                  ...                   http://www.ebi.ac.uk/efo/EFO_0000408
4                                   Age at menopause                  ...                   http://www.ebi.ac.uk/efo/EFO_0001444

[5 rows x 5 columns]
Index(['Disease trait', 'EFO term', 'EFO URI', 'Parent term', 'Parent URI'], dtype='object')


In [14]:
# extend to 250bp
def extend_bed(bed):
    res = bed.replace(".bed",'_extended.bed')
    df = pd.read_table(bed, header=None)
    df['s'] = df.iloc[:,1] - 250
    df['e'] = df.iloc[:,1] + 251
    df['chr'] = df.iloc[:,0]
    df = df[['chr','s','e']].drop_duplicates()
    df.to_csv(res, sep="\t", header=False, index=False)
    
# control set
file_list = glob.glob("SNPsnap_*/matched_snps.bed")
for x in file_list:
    extend_bed(x)
# m6a set
file_list = glob.glob("../*.bed")
for x in file_list:
    extend_bed(x)

In [44]:
#### enrichment analysis
# gwas_dir = "/data5/galaxy/project/GWAS_db/04_class_snp/cluster_by_map/snp_classification/hg19/reformat_bed/"
# gwas_list = glob.glob("%s/*.bed" % gwas_dir)
gwas = "/home/database/GWAS/gwas_hg19.bed2"
##
query_dir = "/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/sig/bed/"
query_list = [x for x in glob.glob("%s/*.bed" % query_dir) if "_extended" not in x]
match_dir="/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/sig/bed/SNPsnap/"

for m6a in query_list:
    sample = os.path.basename(m6a).split(".bed")[0]  #####
    m6a_num = int(os.popen("wc -l %s"%m6a).read().split()[0])
    con = match_dir + os.sep + "SNPsnap_%s/matched_snps.bed" % sample # _extended
    con_num = int(os.popen("wc -l %s"%con).read().split()[0])
    #for gwas in gwas_list:
    overlap_m6a=os.popen("bedtools intersect -a %s -b %s -wa|sort -u|wc -l" % (gwas,m6a)).read().split()[0]
    overlap_con=os.popen("bedtools intersect -a %s -b %s -wa|sort -u|wc -l" % (gwas,con)).read().split()[0]
    a,b,c,d = int(overlap_m6a), int(overlap_con), m6a_num-int(overlap_m6a), con_num-int(overlap_con)
    oddsratio,pvalue = stats.fisher_exact([[a,b], [c,d]])
#     print("####          ASM   Control")
#     print("# GWAS         %d       %d  " % (a,b))
#     print("# Non-GWAS     %d       %d  " % (c,d))
#     print("#############################\n\n")
    print("\t".join([os.path.basename(m6a), str(pvalue), str(oddsratio), str(a), str(b)]))

CRR055525.bed	9.11921145659114e-05	1.8277939008660402	164	54
CRR042298.bed	0.12282649912079449	1.411764705882353	66	34
kid_IP_4.bed	1.2948818792760652e-07	3.074121451670859	86	25
CRR042294.bed	8.076951587777668e-05	2.6041088284286507	70	20
brain_IP_1.bed	0.011529160404608337	1.536189069423929	100	56
CRR073016.bed	0.3953534938064962	1.274511008704557	47	25
mus_IP_5.bed	1.5924089295216472e-08	3.0180996551314068	103	30
CRR042286.bed	0.36558013163965697	1.1874876183683982	82	47
CRR042320.bed	0.03691742782560466	1.9001721170395869	40	15
CRR042292.bed	0.002182105548703262	1.8830211585154353	86	31
CRR055543.bed	0.15358503164431253	1.4968285131767727	45	19
mus_IP_4.bed	0.0005497132744664217	3.2050284629981025	37	10
CRR042282.bed	0.000499886182956333	2.0813308293140977	87	28
CRR073020.bed	0.04725751698560339	2.33839897260274	25	8
CRR042302.bed	3.2563607571676624e-05	2.8843373493975903	63	18
CRR055539.bed	0.047671976774004365	1.9661616161616162	34	12
heart_IP_1.bed	2.0279786939987358e-07	4.08584

In [35]:
# allele-specific m6A
os.chdir("/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/sig/bed/")
result_dir = "/home/galaxy/project/alleleSpecific_analysis/results/GWAS/"
#
gwas = "/home/database/GWAS/gwas_hg19.bed2"
disease = "/home/database/GWAS/gwas_disease.txt"
df = pd.read_table(disease)
rs_dict = dict(zip(df['SNPS'],df['DISEASE/TRAIT']))
# gwas_dir = "/data5/galaxy/project/GWAS_db/04_class_snp/cluster_by_map/snp_classification/hg19/reformat_bed/"
# gwas_list = glob.glob("%s/*.bed" % gwas_dir)
m6a_list = [x for x in glob.glob("*.bed") if "_extended" not in x]
for m6a in m6a_list:#
    # print("############################")
    #for gwas in gwas_list:
#     df = pd.read_table(m6a, sep="\t", header=None)
#     df['m6a'] = df.iloc[:,0] + ":" + df.iloc[:,2].astype(str)
#     m6a_list = df['m6a'].tolist()
    # overlap_m6a=os.popen("bedtools intersect -a %s -b %s -wa|sort -u|wc -l" % (m6a,gwas)).read().split()[0]
    tmp_file = os.path.join(result_dir, 'tmp.txt')
    os.system("bedtools intersect -a %s -b %s -wa|sort -u > %s" % (gwas,m6a,tmp_file))
    rs_list = pd.read_table(tmp_file,header=None).iloc[:,3].tolist()
    result_file = os.path.join(result_dir, os.path.basename(m6a))
    with open(result_file, 'w') as fw:
        lines = ["%s\t%s\n" % (rs, rs_dict[rs]) for rs in rs_list if rs in rs_dict]
        fw.writelines(lines)
os.system("rm %s" % tmp_file)
    # print(os.path.basename(m6a), os.path.basename(gwas), overlap_m6a)

rs284860
rs284860
rs2478839
rs17731
rs943037
rs284860
rs2478839
rs943037
rs10786736
rs3739998
rs11190141
rs4074536
rs4917833
rs284860
rs10786736
rs10786736
rs1127687
rs6892
rs284860
rs10786736
rs1127687
rs10786736
rs284860
rs10786736
rs284860
rs10786736
rs2478839
rs6586145
rs7332
rs7908745
rs284860
rs10786736
rs3739998
rs1127687
rs284860
rs7908745
rs284860
rs284860
rs284860
rs284860
rs284860
rs2296887
rs284860
rs2297991
rs1061810
rs17731
rs284860
rs2297991
rs10786736
rs284860
rs2478839
rs1127687
rs1801253
rs10786736
rs943037
rs2296887
rs1129555
rs2478839
rs1127687
rs284860
rs2296887
rs17731
rs284860
rs10786736
rs284860
rs943037


0

In [32]:
#####  pre-process gwas raw
os.chdir("/home/database/GWAS/")
df = pd.read_table("gwas.tsv")
# cols = ['CHR_ID', 'CHR_POS','CHR_POS','SNPS','P-VALUE','DISEASE/TRAIT','REPORTED GENE(S)']
# df_sub = df[cols]
# df_sub.columns = ['CHR_ID', 'CHR_S','CHR_E','SNPS','P-VALUE','DISEASE/TRAIT','REPORTED GENE(S)']
# df_sub['term'] = df['SNPS']+";"+df['P-VALUE'].astype(str)+';'+df['DISEASE/TRAIT']+';'+df['REPORTED GENE(S)']
# df_res = df_sub[['CHR_ID','CHR_S','CHR_E','term']]
# df_res.to_csv("GWAS_hg38.bed", sep="\t", header=False, index=False)
df['CHR_ID'] = "chr" + df['CHR_ID'].astype(str)
df_sub = df[['CHR_ID', 'CHR_POS','CHR_POS','SNPS','DISEASE/TRAIT']]
df_sub.columns = ['CHR_ID','CHR_S','CHR_E','SNPS','DISEASE/TRAIT']
df_sub['CHR_ID'] = df_sub['CHR_ID'].str.split(";").str[0]
df_sub['CHR_S'] = df_sub['CHR_S'].str.split(";").str[0]
df_sub['CHR_E'] = df_sub['CHR_E'].str.split(";").str[0]
df_sub['SNPS'] = df_sub['SNPS'].str.split(";").str[0]
df_sub['DISEASE/TRAIT'] = df_sub['DISEASE/TRAIT'].str.split(";").str[0]
df_sub = df_sub.dropna()
df_sub['CHR_ID'] = df_sub['CHR_ID'].str.split(" x").str[0]
df_sub['CHR_S'] = df_sub['CHR_S'].str.split(" x").str[0]
df_sub['CHR_E'] = df_sub['CHR_E'].str.split(" x").str[0]
df_sub['SNPS'] = df_sub['SNPS'].str.split(" x").str[0]
df_sub['DISEASE/TRAIT'] = df_sub['DISEASE/TRAIT'].str.split(" x").str[0]
# df_sub.to_csv("gwas_hg38.bed", sep="\t", header=False, index=False)
df_sub = df_sub[~df_sub['SNPS'].str.contains(',')]
df_sub['CHR_S'] = df_sub['CHR_S'].astype(int) - 1
df_map = df_sub[['SNPS', 'DISEASE/TRAIT']]
df_map.to_csv("gwas_disease.txt", sep="\t", index=False)
df_sub.to_csv("gwas_hg38.bed", sep="\t", header=False, index=False)

/home/galaxy/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11,12,23,27,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/galaxy/an